# Azure AI Agents 支持模型上下文协议 (MCP) - Python

本笔记本展示了如何在 Python 中使用支持模型上下文协议 (MCP) 的 Azure AI Agents。它演示了如何创建一个智能代理，通过无密钥认证利用外部 MCP 服务器（例如 Microsoft Learn）来增强功能。


## 安装所需的 Python 包

首先，我们需要安装必要的 Python 包：
- **azure-ai-projects**：核心 Azure AI Projects SDK
- **azure-ai-agents**：用于创建和管理代理的 Azure AI Agents SDK
- **azure-identity**：通过 DefaultAzureCredential 提供无密钥认证
- **mcp**：Python 的模型上下文协议实现


## 无密钥认证的优势

本笔记本演示了**无密钥认证**，它具有以下几个优势：
- ✅ **无需管理 API 密钥** - 使用基于 Azure 身份的认证
- ✅ **增强的安全性** - 代码或配置文件中无需存储任何密钥
- ✅ **自动凭据轮换** - Azure 负责凭据生命周期管理
- ✅ **基于角色的访问控制** - 使用 Azure RBAC 实现精细化权限管理
- ✅ **多环境支持** - 在开发和生产环境中无缝运行

`DefaultAzureCredential` 会自动选择最佳可用的凭据来源：
1. **托管身份**（在 Azure 中运行时）
2. **Azure CLI** 凭据（本地开发时）
3. **Visual Studio** 凭据
4. **环境变量**（如果已配置）
5. **交互式浏览器**认证（作为备用方案）


## 无密钥认证设置

**无密钥认证的前提条件：**

### 本地开发：
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Azure 环境：
- 在您的 Azure 资源上启用 **系统分配的托管身份**
- 为托管身份分配适当的 **RBAC 角色**：
  - `Cognitive Services OpenAI User` 用于访问 Azure OpenAI
  - `AI Developer` 用于访问 Azure AI 项目

### 环境变量（可选）：
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**无需 API 密钥或连接字符串！** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## 导入所需库

导入必要的 Python 模块：
- **os, time**：用于环境变量和延迟的标准 Python 库
- **AIProjectClient**：Azure AI Projects 的主要客户端
- **DefaultAzureCredential**：用于 Azure 服务的无密钥认证
- **与 MCP 相关的类**：用于创建和管理 MCP 工具以及处理审批


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## 配置 MCP 服务器设置

使用环境变量设置 MCP 服务器配置，并提供默认值作为后备选项：
- **MCP_SERVER_URL**：MCP 服务器的 URL（默认为 Microsoft Learn API）
- **MCP_SERVER_LABEL**：用于标识 MCP 服务器的标签（默认为 "mslearn"）

这种方法可以在不同环境中实现灵活配置。


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## 创建 Azure AI 项目客户端（无密钥认证）

使用**无密钥认证**初始化 Azure AI 项目客户端：
- **endpoint**：Azure AI Foundry 项目的端点 URL
- **credential**：使用 `DefaultAzureCredential()` 实现安全的无密钥认证
- **无需 API 密钥**：自动发现并使用最佳可用凭据

**认证流程：**
1. 检查托管身份（在 Azure 环境中）
2. 回退到 Azure CLI 凭据（用于本地开发）
3. 根据需要使用其他可用的凭据来源

这种方法无需在代码中管理 API 密钥或连接字符串。


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## 创建 MCP 工具定义

创建一个 MCP 工具，用于连接到 Microsoft Learn MCP 服务器：
- **server_label**：MCP 服务器的标识符
- **server_url**：MCP 服务器的 URL 端点
- **allowed_tools**：可选列表，用于限制可以使用的工具（空列表允许所有工具）

此工具将使代理能够访问 Microsoft Learn 文档和资源。


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## 创建代理并执行对话（无密钥工作流）

本节全面展示了完整的**无密钥代理工作流**：

1. **创建 AI 代理**：使用 GPT-4.1 nano 模型和 MCP 工具设置代理
2. **创建线程**：建立用于交流的对话线程
3. **发送消息**：询问代理关于 Azure OpenAI 与 OpenAI 的区别
4. **处理工具审批**：在需要时自动批准 MCP 工具调用
5. **监控执行**：跟踪代理的进度并处理任何必要的操作
6. **显示结果**：展示对话内容和工具使用详情

**无密钥功能：**
- ✅ **无硬编码密钥** - 所有身份验证由 Azure 身份处理
- ✅ **默认安全** - 使用基于角色的访问控制
- ✅ **简化部署** - 无需凭证管理
- ✅ **审计友好** - 所有访问均通过 Azure 身份进行跟踪

代理将使用 MCP 工具访问 Microsoft Learn 资源，确保完全安全且无需 API 密钥管理。


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**免责声明**：  
本文档使用AI翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 进行翻译。尽管我们努力确保翻译的准确性，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文档应被视为权威来源。对于关键信息，建议使用专业人工翻译。我们不对因使用此翻译而产生的任何误解或误读承担责任。
